In [1]:
import pandas as pd
import requests
import json

In [2]:
df = pd.read_parquet('article.parquet')

In [4]:
apikey = '1d5898219f7c5d3de72e543775524cb5'
url = 'https://api.meaningcloud.com'
endpoint = '/sentiment-2.1'

In [11]:
parameters = {
    'key': apikey,
    'lang': 'pt',
    'txt': '2023 é ano com mais ataques de tubarão desde 2006 em Pernambuco'
}

In [12]:
x = requests.post(url + endpoint, params = parameters)

In [13]:
x.json()

{'agreement': 'AGREEMENT',
 'confidence': '100',
 'irony': 'NONIRONIC',
 'model': 'general_pt',
 'score_tag': 'N',
 'sentence_list': [{'agreement': 'AGREEMENT',
   'bop': 'y',
   'confidence': '100',
   'endp': '62',
   'inip': '0',
   'score_tag': 'N',
   'segment_list': [{'agreement': 'AGREEMENT',
     'confidence': '100',
     'endp': '62',
     'inip': '0',
     'polarity_term_list': [{'confidence': '100',
       'endp': '26',
       'inip': '20',
       'score_tag': 'N',
       'sentimented_entity_list': [{'endp': '62',
         'form': 'Pernambuco',
         'id': '4a3031d810',
         'inip': '53',
         'score_tag': 'N',
         'type': 'Top>Location>GeoPoliticalEntity>Adm1',
         'variant': 'Pernambuco'}],
       'text': 'ataque@N'},
      {'confidence': '100',
       'endp': '37',
       'inip': '31',
       'score_tag': 'N',
       'sentimented_entity_list': [{'endp': '62',
         'form': 'Pernambuco',
         'id': '4a3031d810',
         'inip': '53',
         '

In [18]:
df['sentiment'] = ''
df['irony'] = ''
df['confidence'] = 0

In [23]:
for index, row in df.iterrows():
    parameters['txt'] = row['title']
    x = requests.post(url + endpoint, params = parameters)
    if x.status_code >= 500:
        time.sleep(1)
        x = requests.get(url + endpoint, params = parameters)
        if x.status_code >= 500:
            raise Exception('internal error ' + str(x.status_code))
    x = x.json()
    df.loc[index, 'sentiment'] = x['score_tag']
    df.loc[index, 'irony'] = x['irony']
    df.loc[index, 'confidence'] = x['confidence']

In [25]:
df

,title,url,image,publishedAt,name,year,month,day,sentiment,irony,confidence
0,2023 é ano com mais ataques de tubarão desde 2...,https://noticias.uol.com.br/colunas/carlos-mad...,https://conteudo.imguol.com.br/c/noticias/d7/2...,2023-03-08 03:00:00,UOL,2023,3,8,N,NONIRONIC,100
1,"Tubarão-branco, tubarão-touro e tubarão-tigre ...",https://noticias.uol.com.br/meio-ambiente/ulti...,https://conteudo.imguol.com.br/c/noticias/59/2...,2023-03-08 03:00:00,UOL,2023,3,8,N+,NONIRONIC,98
2,Tubarão: Placas de alerta para ataques estão d...,https://www1.folha.uol.com.br/cotidiano/2023/0...,https://f.i.uol.com.br/fotografia/2023/03/07/1...,2023-03-07 20:52:00,Folha de S.Paulo,2023,3,8,N,NONIRONIC,100
3,Por que ataques de tubarão são mais comuns em ...,https://www.tnh1.com.br/noticia/nid/por-que-at...,https://www.tnh1.com.br/fileadmin/_processed_/...,2023-03-07 18:14:03,TNH1,2023,3,8,N,NONIRONIC,100
4,"Em 16 dias, menina é a 3ª vítima atacada por t...",https://www.tnh1.com.br/noticia/nid/em-16-dias...,https://www.tnh1.com.br/fileadmin/_processed_/...,2023-03-07 16:16:08,TNH1,2023,3,8,N,NONIRONIC,100
5,Por que ataques de tubarão são mais comuns em ...,https://www.brasil247.com/regionais/nordeste/p...,https://publisher-publish.s3.eu-central-1.amaz...,2023-03-07 15:15:05,Brasil 247,2023,3,8,N,NONIRONIC,100
6,Região de Recife registra três casos de ataque...,https://www.ibahia.com/brasil/regiao-do-grande...,https://www.ibahia.com/wp-content/uploads/2023...,2023-03-07 12:50:19,iBahia,2023,3,8,N,NONIRONIC,100
7,Proibido cores néon e joias: laboratório em Pe...,https://extra.globo.com/noticias/brasil/proibi...,https://extra.globo.com/incoming/13030292-1a3-...,2023-03-07 07:00:32,Extra,2023,3,8,N,NONIRONIC,100
8,"No PV, Ferroviário tenta encaminhar classifica...",https://www.opovo.com.br/esportes/futebol/copa...,https://www.opovo.com.br/_midias/jpg/2023/02/2...,2023-03-07 03:00:00,O POVO,2023,3,8,N,NONIRONIC,100
9,Ataque de tubarão: Por que tem tantos animais ...,https://www.opovo.com.br/noticias/brasil/2023/...,https://www.opovo.com.br/_midias/jpg/2023/03/0...,2023-03-07 03:00:00,O POVO,2023,3,8,N,NONIRONIC,100
